In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import shuffle
import glob
from sklearn.preprocessing import LabelEncoder as le
from sklearn.model_selection import train_test_split
from keras.models import load_model

Using TensorFlow backend.


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.applications import MobileNet,densenet
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import concatenate, add
from keras.layers import Input


In [3]:
# cla = [x[1] for x in os.walk("../input/vision_asgn2/vision_asgn2")][0]

In [4]:
data = []
y = []
fine = True
kk = glob.glob('../input/visreca2/train/train/*/*/*.jpg')
shuffle(kk)
for i in kk:
    data.append(cv2.resize(mpimg.imread(i)/255.0 ,(224,224)))
    pp = i.split('/')
    if fine == True:
        y.append(pp[5] + '_' + pp[6])
    else:
        y.append(pp[5])
data = np.array(data)

# kk


In [5]:
set(y)

{'aircrafts_1',
 'aircrafts_2',
 'aircrafts_3',
 'aircrafts_4',
 'aircrafts_5',
 'aircrafts_6',
 'aircrafts_7',
 'birds_1',
 'birds_10',
 'birds_11',
 'birds_2',
 'birds_3',
 'birds_4',
 'birds_5',
 'birds_6',
 'birds_7',
 'birds_8',
 'birds_9',
 'cars_1',
 'cars_2',
 'cars_3',
 'cars_4',
 'cars_5',
 'cars_6',
 'cars_7',
 'cars_8',
 'dogs_1',
 'dogs_2',
 'dogs_3',
 'dogs_4',
 'dogs_5',
 'flowers_1',
 'flowers_2',
 'flowers_3',
 'flowers_4',
 'flowers_5'}

In [6]:
le_ = le()
le_.fit(y)
y = le_.transform(y)

In [7]:
le_name_mapping = dict(zip(le_.transform(le_.classes_) , le_.classes_ ))
print(le_name_mapping)

{0: 'aircrafts_1', 1: 'aircrafts_2', 2: 'aircrafts_3', 3: 'aircrafts_4', 4: 'aircrafts_5', 5: 'aircrafts_6', 6: 'aircrafts_7', 7: 'birds_1', 8: 'birds_10', 9: 'birds_11', 10: 'birds_2', 11: 'birds_3', 12: 'birds_4', 13: 'birds_5', 14: 'birds_6', 15: 'birds_7', 16: 'birds_8', 17: 'birds_9', 18: 'cars_1', 19: 'cars_2', 20: 'cars_3', 21: 'cars_4', 22: 'cars_5', 23: 'cars_6', 24: 'cars_7', 25: 'cars_8', 26: 'dogs_1', 27: 'dogs_2', 28: 'dogs_3', 29: 'dogs_4', 30: 'dogs_5', 31: 'flowers_1', 32: 'flowers_2', 33: 'flowers_3', 34: 'flowers_4', 35: 'flowers_5'}


In [8]:
import pickle
pickle_out = open("dict_f_20_early_stopping_90.pickle","wb")
pickle.dump(le_name_mapping, pickle_out)
pickle_out.close()

In [9]:
x_train,x_test,y_train_,y_test_ = train_test_split(data,y,test_size = .1)

In [10]:
y_train_ = keras.utils.to_categorical(y_train_, 36)
y_test_ = keras.utils.to_categorical(y_test_, 36)

In [11]:
x_train.shape

(2180, 224, 224, 3)

In [12]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=30,
    horizontal_flip=True,
    zoom_range=0.2)
datagen.fit(x_train)

In [13]:
'Done'

'Done'

In [14]:
inp = Input(shape = (224,224,3))

In [15]:
foo = glob.glob('../input/*/*.h5')[0]

In [16]:
#Course 
model_C = load_model(foo)
Cx = model_C(inp)


In [17]:
#19# model 2

base_model1=MobileNet(weights='imagenet',include_top=False) 
x=base_model1(inp)
x1=GlobalAveragePooling2D()(x)
# x=Dense(512,activation='relu')(x) 
# preds=Dense(5,activation='softmax')(x) 
#model1 = Model(inputs=base_model1.input, outputs=x1)
#model1.summary()

/opt/conda/lib/python3.6/site-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


17227776/17225924 [==============================] - 0s 0us/step


In [18]:

base_model2 = densenet.DenseNet121(include_top = False, weights='imagenet')

x=base_model2(inp)
x2=GlobalAveragePooling2D()(x)
#model2=Model(inputs=base_model2.input,outputs=x2)
# len(model.layers)
#model2.summary()

29089792/29084464 [==============================] - 0s 0us/step


In [19]:
concatenated = concatenate([x1, x2, Cx])
concat_out = Dense(1024, activation='relu')(concatenated)
#concat_out = Dense(1024,activation = 'relu')(concat_out)
concat_out_ = Dense(36, activation='softmax')(concat_out)
merged_model= Model(inputs=inp, outputs=concat_out_)

merged_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
mobilenet_1.00_224 (Model)      multiple             3228864     input_1[0][0]                    
__________________________________________________________________________________________________
densenet121 (Model)             multiple             7037504     input_1[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 1024)         0           mobilenet_1.00_224[1][0]         
__________________________________________________________________________________________________
global_ave

In [20]:
'Done'

'Done'

In [21]:
len(merged_model.layers)

9

In [22]:
merged_model.layers[7:]

In [23]:
model = merged_model

In [24]:
# m= Merge([base_model1, base_model2,], mode='concat')

In [25]:
# model=Model(inputs=base_model.input,outputs=preds)
# len(model.layers)

In [26]:
for layer in model.layers[:7]:
    layer.trainable=False
for layer in model.layers[7:]:
    layer.trainable=True
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
mobilenet_1.00_224 (Model)      multiple             3228864     input_1[0][0]                    
__________________________________________________________________________________________________
densenet121 (Model)             multiple             7037504     input_1[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 1024)         0           mobilenet_1.00_224[1][0]         
__________________________________________________________________________________________________
global_ave

In [27]:
optimizer = Adam(lr = 0.001, decay=0.05, amsgrad=True)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [28]:
callbacks = [EarlyStopping(monitor = 'val_loss', patience = 7)]

In [29]:
model.fit_generator(datagen.flow(x_train,y_train_,batch_size = 64),
                    steps_per_epoch= len(x_train)/64,
                    epochs = 100,
                    callbacks=callbacks,
                    validation_data = (x_test,y_test_))

Epoch 1/100
35/34 [==============================] - 59s 2s/step - loss: 1.3279 - acc: 0.6446 - val_loss: 0.4297 - val_acc: 0.8642
Epoch 2/100
35/34 [==============================] - 31s 887ms/step - loss: 0.4319 - acc: 0.8664 - val_loss: 0.3105 - val_acc: 0.9053
Epoch 3/100
35/34 [==============================] - 35s 990ms/step - loss: 0.3188 - acc: 0.9028 - val_loss: 0.2678 - val_acc: 0.9383
Epoch 4/100
35/34 [==============================] - 35s 1000ms/step - loss: 0.2565 - acc: 0.9242 - val_loss: 0.2618 - val_acc: 0.9424
Epoch 5/100
35/34 [==============================] - 35s 1s/step - loss: 0.2035 - acc: 0.9515 - val_loss: 0.2114 - val_acc: 0.9506
Epoch 6/100
35/34 [==============================] - 35s 1s/step - loss: 0.1871 - acc: 0.9506 - val_loss: 0.2041 - val_acc: 0.9547
Epoch 7/100
35/34 [==============================] - 36s 1s/step - loss: 0.1614 - acc: 0.9616 - val_loss: 0.2005 - val_acc: 0.9465
Epoch 8/100
35/34 [==============================] - 35s 1s/step - loss: 

In [30]:
np.argmax(model.predict(x_test,axis=1)

SyntaxError: unexpected EOF while parsing (<ipython-input-30-a855c5f32397>, line 1)

In [31]:
np.argmax(y_test_,axis=1)

array([26,  6, 34, 30, 31,  1, 31, 34, 25,  0, 34, 35,  0, 28, 32,  6, 34,
       26, 32,  6, 18, 30, 17, 30, 27, 30,  0, 30, 25, 33,  1,  0,  0,  8,
        1,  0, 31,  0, 30, 26, 27, 34, 30,  0, 34, 26, 28, 27, 24, 24, 23,
       34, 24, 19,  1,  3,  2,  0, 33, 31, 30, 26, 24, 19,  0,  0, 28, 10,
       19, 35, 34, 31, 18, 26,  4, 33, 35, 20, 33, 17, 34, 27,  0,  2,  6,
       30, 28, 32, 30, 34,  4,  2, 15, 29, 18, 30, 33, 29, 26, 27, 29,  5,
       12, 32, 22, 10, 30, 28, 14,  0, 32, 28, 27, 34, 24, 22,  5,  0, 32,
       28, 26, 18, 33, 30, 29,  5, 33,  0, 29,  0,  6,  2,  2,  0, 12, 29,
        0, 35,  2, 33, 28, 14, 18,  4, 28, 19, 15, 31, 33, 17, 27, 21, 32,
       34,  0, 29, 33, 33,  0, 33, 33,  0, 33,  6, 33, 11, 21,  0, 20, 32,
       27, 11, 35, 33, 33, 28, 30, 29, 23,  1, 33, 34, 19, 31, 26, 35, 29,
       31,  0, 27,  2,  0,  5, 27, 30, 31,  0, 26, 33, 33, 29, 29, 22, 14,
        6, 35, 32,  1,  1,  2,  7, 34, 32, 35,  4, 34, 16, 33, 17, 27, 29,
       35, 14, 29, 32, 33

In [32]:
model.save('FINE_20_early_stopping_90.h5')